# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import epyestim
import epyestim.covid19 as covid19

from fbprophet import Prophet

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import itertools
import statsmodels.api as sm

from statsmodels.tsa.stattools import acf
import pmdarima as pm

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

# Load Data

In [ ]:
dados_covid = pd.read_parquet('../data/processed/covid_saude.parquet')
dados_covid['data'] = pd.to_datetime(dados_covid['data'])
dados_covid.head()

In [ ]:
df_vacinacao = pd.read_parquet('../data/processed/vacinacao_diaria_SP.parquet')

In [ ]:
df_vacinacao = df_vacinacao.loc[df_vacinacao['paciente_endereco_nmmunicipio']=='SAO PAULO']

In [ ]:
df_vacinacao_first = df_vacinacao.loc[df_vacinacao['vacina_descricao_dose'].isin(['1ª\xa0Dose'])]
df_vacinacao_second = df_vacinacao.loc[df_vacinacao['vacina_descricao_dose'].isin(['2ª\xa0Dose'])]

## Separando a base de COVID entre BR, Estadual e Municipal

**Dados COVID**  
```
dados_covid_brasil  
dados_covid_estados  
dados_covid_cidades 
```

In [ ]:
dados_covid.sort_values(by=['data'], inplace=True)

In [ ]:
dados_covid['year'] = dados_covid['data'].dt.year
dados_covid['weekday'] = dados_covid['data'].dt.weekday
dados_covid['week_number'] = 0
dados_covid.loc[dados_covid['weekday']==0, 'week_number'] = 1

In [ ]:
dados_covid_cidades = dados_covid.loc[dados_covid['regiao']!='Brasil']
dados_covid_cidades = dados_covid_cidades[dados_covid_cidades['municipio'].notna()]
dados_covid_cidades.reset_index(drop=True, inplace=True)

In [ ]:
dados_covid_estados = dados_covid.loc[dados_covid['regiao']!='Brasil']
dados_covid_estados = dados_covid_estados[dados_covid_estados['municipio'].isna()]
dados_covid_estados.reset_index(drop=True, inplace=True)

In [ ]:
dados_covid_brasil = dados_covid.loc[dados_covid['regiao']=='Brasil']
dados_covid_brasil.reset_index(drop=True, inplace=True)

In [ ]:
dados_covid_cidades['week_number'] = dados_covid_cidades.groupby('codmun')['week_number'].cumsum()
dados_covid_estados['week_number'] = dados_covid_estados.groupby('estado')['week_number'].cumsum()
dados_covid_brasil['week_number'] = dados_covid_brasil['week_number'].cumsum()

## São Paulo Exploration

In [ ]:
df_sp = dados_covid_cidades.loc[dados_covid_cidades['municipio']=='São Paulo']

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(data=df_sp.groupby('week_number').sum(), x='week_number', y='obitosNovos')
plt.show();

### Grouping by epidemiological week

In [ ]:
df_sp_week = df_sp.groupby('week_number').sum()
df_sp_week['est_r'] = df_sp_week['obitosNovos'].divide(df_sp_week['obitosNovos'].shift(1))
df_sp_week.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(data=df_sp_week, x='week_number', y='obitosNovos')
plt.show();

In [ ]:
#adicionando infos de habitantes
df_sp_week['populacao'] = 45919049
df_sp_week['casos_p_habitante'] = df_sp_week['obitosNovos'] / df_sp_week['populacao']
df_sp_week['casos_p_milhao'] = df_sp_week['obitosNovos'] / (df_sp_week['populacao'] / 1000000)

##Calculando a Taxa de Reprodução estimada (Re)
Utilizand método Bayesiano como citado nas referências. O estad do Espírito Santo possui alguma incongruência nos dados e foi deixado de fora.

In [ ]:
#calculando o R estimado por dia
data_sp = df_sp[['data', 'obitosNovos']]
data_sp.set_index('data', inplace=True)
r_est_sp= covid19.r_covid(data_sp['obitosNovos'], r_window_size=3)

In [ ]:
#checando os dfs gerados

print(r_est_sp.head())
print('\n')

###Adicionando ao df

In [ ]:
r_est_sp = r_est_sp.reset_index().rename(columns={'index' : 'data'})
df_sp = df_sp.merge(r_est_sp, on='data')

#Plotando o R estimado para São Paulo, Rio de Janeiro e Minas Gerais
Dataframes:


```
dados_covid_sudeste_mg
dados_covid_sudeste_sp
dados_covid_sudeste_ej
```



Como podemos ver nos gráficos abaixo, picos de Re maiores que 1,2 são suscedidos por picos de infectados. Dessa forma, podemos usar o pico de Re > 1,2 para prever picos de infectados no período seguinte.

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.lineplot(data=df_sp, x='data', y='R_mean')
plt.title('R médio para o estado de São Paulo')
plt.xlabel('Dia')
plt.ylabel('R médio')
ax.axhline(1, ls='--')
ax;

df = df_sp.groupby('data').sum()
df['casosNovos_mediamovel'] = df['casosNovos'].rolling(7).mean() #criando as médias móveis

plt.figure(figsize=(12,6))
ax = sns.lineplot(data=df, x='data', y='casosNovos_mediamovel')
plt.title('Casos novos (média móvel de 7 dias) para o estado de São Paulo')
plt.xlabel('Dia')
plt.ylabel('Casos Novos')
ax;

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    actual_non_zero = actual[actual!=0]
    forecast_non_zero = forecast.loc[forecast.index & actual_non_zero.index]
    
    mape = np.mean(np.abs(forecast_non_zero - actual_non_zero)/np.abs(actual_non_zero))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast_non_zero - actual_non_zero)/actual_non_zero)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(forecast-actual)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

# ARIMA

## São Paulo

In [ ]:
decomposicao = seasonal_decompose(df_sp[['data', 'obitosNovos']].set_index('data'))
decomposicao

In [ ]:
decomposicao.plot(); #o erro foi menor no modelo aditivo

In [ ]:
df_sp.sort_values(by='data', inplace=True)

In [ ]:
df_sp['week_number'].describe()

## Predição diária

In [ ]:
dados_treino = df_sp.loc[df_sp['week_number']<75]
dados_teste = df_sp.loc[df_sp['week_number']>=75]

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=df_sp, x='data', y='obitosNovos')
plt.show();

In [ ]:
plot_acf(df_sp['obitosNovos']); #entendendo autocorrelação

In [ ]:
plot_pacf(df_sp['obitosNovos']); #entendendo autocorrelação parcial

In [ ]:
dict_metrics = {
    'p':[],
    'd':[],
    'q':[],
    'mape':[],
    'me':[],
    'mae':[],
    'mpe':[],
    'rmse':[],
    'acf1':[],
    'corr':[],
    'minmax':[]
}

i = 0

for p in tqdm(range(10)):
    for d in range(10):
        for q in range(10):
            try:
                modelo_arima = ARIMA(dados_treino['obitosNovos'], order=[p,d,q])
                modelo_arima_treinado = modelo_arima.fit()

                # Forecast
                fc = modelo_arima_treinado.forecast(dados_teste.shape[0], alpha=0.05)  # 95% conf

                metrics = forecast_accuracy(fc, dados_teste['obitosNovos'])

                dict_metrics['p'].append(p)
                dict_metrics['d'].append(d)
                dict_metrics['q'].append(q)
                dict_metrics['mape'].append(metrics['mape'])
                dict_metrics['me'].append(metrics['me'])
                dict_metrics['mae'].append(metrics['mae'])
                dict_metrics['mpe'].append(metrics['mpe'])
                dict_metrics['rmse'].append(metrics['rmse'])
                dict_metrics['acf1'].append(metrics['acf1'])
                dict_metrics['corr'].append(metrics['corr'])
                dict_metrics['minmax'].append(metrics['minmax'])

            except Exception as e:
                print('Error:', e, 'p:', p, 'd:', d, 'q:', q)

In [ ]:
correls = np.array(dict_metrics['corr'])
mapes = np.array(dict_metrics['mape'])
rmses = np.array(dict_metrics['rmse'])
maes = np.array(dict_metrics['mae'])
list_p = np.array(dict_metrics['p'])
list_d = np.array(dict_metrics['d'])
list_q = np.array(dict_metrics['q'])

nan_indexes = np.argwhere(np.isnan(correls))
correls = np.delete(correls, nan_indexes, axis=0)
mapes = np.delete(mapes, nan_indexes, axis=0)
rmses = np.delete(rmses, nan_indexes, axis=0)
maes = np.delete(maes, nan_indexes, axis=0)
list_p = np.delete(list_p, nan_indexes, axis=0)
list_d = np.delete(list_d, nan_indexes, axis=0)
list_q = np.delete(list_q, nan_indexes, axis=0)

inv_mapes = np.reciprocal(mapes)
inv_rmses = np.reciprocal(rmses)
inv_maes = np.reciprocal(maes)

result = np.multiply(np.multiply(np.multiply(inv_mapes,correls), inv_rmses), inv_maes)

In [ ]:
max_arg = result.argmax(axis=0)
max_arg

In [ ]:
dict_metrics.keys()

In [ ]:
p_max = list_p[max_arg]
d_max = list_d[max_arg]
q_max = list_q[max_arg]

In [ ]:
p_max # 9

In [ ]:
d_max # 1

In [ ]:
q_max # 2

In [ ]:
# modelo_arima = ARIMA(dados_treino['obitosNovos'].values.astype('float32'), order=[p_max,d_max,q_max])
modelo_arima = ARIMA(dados_treino['obitosNovos'].values.astype('float32'), order=[9,1,2])
modelo_arima_treinado = modelo_arima.fit()
forecast = modelo_arima_treinado.get_forecast(dados_teste.shape[0])

fc = modelo_arima_treinado.forecast(dados_teste.shape[0], alpha=0.05)
dados_teste['Previsao'] = fc
# dados_teste['Previsao'] = forecast.predicted_mean

# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()

# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_treino.loc[dados_treino['data']>='2021-06-01', 'obitosNovos'], label="Train")
plt.plot(dados_teste['obitosNovos'], color='orange', label="Test Real")
plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

# plt.figure(figsize=(12,6))
# sns.lineplot(data=dados_treino[250:], x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='Previsao')

In [ ]:
forecast_accuracy(dados_teste['Previsao'], dados_teste['obitosNovos'])

### Predicting all dates

In [ ]:
confidence_intervals = np.empty((0,2), int)
predictions = []

for data in df_sp.loc[df_sp['data']>='2020-05-01', 'data'].sort_values():
    print(data)
    dados_treino = df_sp.loc[df_sp['data']<data]
    dados_teste = df_sp.loc[df_sp['data']==data]
    
#     modelo_arima = ARIMA(dados_treino['obitosNovos'].values, order=[p_max,d_max,q_max])
    modelo_arima = ARIMA(dados_treino['obitosNovos'].values, order=[9,1,2])
    modelo_arima_treinado = modelo_arima.fit()
    forecast = modelo_arima_treinado.get_forecast(dados_teste.shape[0])

    fc = modelo_arima_treinado.forecast(dados_teste.shape[0], alpha=0.05)
    predictions.append(fc[0])
    # dados_teste['Previsao'] = forecast.predicted_mean

    # Get confidence intervals of forecasts
    confidence_intervals = np.concatenate((confidence_intervals,[forecast.conf_int()[0]]))

# # make series for plotting purpose
# lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
# upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# # Plot
# plt.figure(figsize=(15,6))
# plt.plot(dados_treino['obitosNovos'], label="Train")
# plt.plot(dados_teste['obitosNovos'], color='orange', label="Test Real")
# plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
# plt.fill_between(lower_series.index, 
#                  lower_series, 
#                  upper_series, 
#                  color='k', alpha=.15)

# plt.legend()
# plt.title("Covid Forecasting - Deaths")
# plt.show()

In [ ]:
dados_teste = df_sp.loc[df_sp['data']>='2020-05-01']

In [ ]:
dados_teste.shape

In [ ]:
confidence_intervals.shape

In [ ]:
# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)
df_predictions = pd.Series(predictions, index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_teste[300:400]['obitosNovos'], color='orange', label="Test Real")
plt.plot(df_predictions[300:400], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series[300:400].index, 
                 lower_series[300:400], 
                 upper_series[300:400], 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

## Predição semanal

In [ ]:
df_sp_wk = df_sp.groupby(['week_number'])['obitosNovos'].sum().reset_index()

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=df_sp_wk, x='week_number', y='obitosNovos')
plt.show();

In [ ]:
plot_acf(df_sp_wk['obitosNovos']); #entendendo autocorrelação

In [ ]:
plot_pacf(df_sp_wk['obitosNovos']); #entendendo autocorrelação

In [ ]:
dados_treino = df_sp_wk.loc[df_sp_wk['week_number']<75]
dados_teste = df_sp_wk.loc[df_sp_wk['week_number']>=75]

In [ ]:
dict_metrics = {
    'p':[],
    'd':[],
    'q':[],
    'mape':[],
    'me':[],
    'mae':[],
    'mpe':[],
    'rmse':[],
    'acf1':[],
    'corr':[],
    'minmax':[]
}

i = 0

for p in tqdm(range(10)):
    for d in range(10):
        for q in range(10):
            try:
                modelo_arima = ARIMA(dados_treino['obitosNovos'], order=[p,d,q])
                modelo_arima_treinado = modelo_arima.fit()

                # Forecast
                fc = modelo_arima_treinado.forecast(dados_teste.shape[0], alpha=0.05)  # 95% conf

                metrics = forecast_accuracy(fc, dados_teste['obitosNovos'])

                dict_metrics['p'].append(p)
                dict_metrics['d'].append(d)
                dict_metrics['q'].append(q)
                dict_metrics['mape'].append(metrics['mape'])
                dict_metrics['me'].append(metrics['me'])
                dict_metrics['mae'].append(metrics['mae'])
                dict_metrics['mpe'].append(metrics['mpe'])
                dict_metrics['rmse'].append(metrics['rmse'])
                dict_metrics['acf1'].append(metrics['acf1'])
                dict_metrics['corr'].append(metrics['corr'])
                dict_metrics['minmax'].append(metrics['minmax'])

            except Exception as e:
                print('Error:', e, 'p:', p, 'd:', d, 'q:', q)

In [ ]:
correls = np.array(dict_metrics['corr'])
mapes = np.array(dict_metrics['mape'])
rmses = np.array(dict_metrics['rmse'])
maes = np.array(dict_metrics['mae'])
list_p = np.array(dict_metrics['p'])
list_d = np.array(dict_metrics['d'])
list_q = np.array(dict_metrics['q'])

nan_indexes = np.argwhere(np.isnan(correls))
correls = np.delete(correls, nan_indexes, axis=0)
mapes = np.delete(mapes, nan_indexes, axis=0)
rmses = np.delete(rmses, nan_indexes, axis=0)
maes = np.delete(maes, nan_indexes, axis=0)
list_p = np.delete(list_p, nan_indexes, axis=0)
list_d = np.delete(list_d, nan_indexes, axis=0)
list_q = np.delete(list_q, nan_indexes, axis=0)

inv_mapes = np.reciprocal(mapes)
inv_rmses = np.reciprocal(rmses)
inv_maes = np.reciprocal(maes)

result = np.multiply(np.multiply(np.multiply(inv_mapes,correls), inv_rmses), inv_maes)

In [ ]:
max_arg = result.argmax(axis=0)
max_arg

In [ ]:
dict_metrics.keys()

In [ ]:
p_max = list_p[max_arg]
d_max = list_d[max_arg]
q_max = list_q[max_arg]

In [ ]:
p_max # 0

In [ ]:
d_max # 1

In [ ]:
q_max # 2

In [ ]:
# modelo_arima = ARIMA(dados_treino['obitosNovos'].values.astype('float32'), order=[p_max,d_max,q_max])
modelo_arima = ARIMA(dados_treino['obitosNovos'].values.astype('float32'), order=[4,1,5])
modelo_arima_treinado = modelo_arima.fit()
forecast = modelo_arima_treinado.get_forecast(dados_teste.shape[0])

fc = modelo_arima_treinado.forecast(dados_teste.shape[0], alpha=0.05)
dados_teste['Previsao'] = fc
# dados_teste['Previsao'] = forecast.predicted_mean

# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()

# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_treino.loc[dados_treino['week_number']>=50, 'obitosNovos'], label="Train")
plt.plot(dados_teste['obitosNovos'], color='orange', label="Test Real")
plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

# plt.figure(figsize=(12,6))
# sns.lineplot(data=dados_treino[250:], x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='Previsao')

In [ ]:
forecast_accuracy(dados_teste['Previsao'], dados_teste['obitosNovos'])

## Média móvel - diário

In [ ]:
dias_media_movel = 7

df_sp['obitosNovosMA'] = df_sp['obitosNovos'].transform(lambda x: x.rolling(dias_media_movel, 1).mean())

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=df_sp, x='data', y='obitosNovosMA')
plt.show();

In [ ]:
plot_acf(df_sp['obitosNovosMA']); #entendendo autocorrelação

In [ ]:
plot_pacf(df_sp['obitosNovosMA']); #entendendo autocorrelação

In [ ]:
dados_treino = df_sp.loc[df_sp['week_number']<75]
dados_teste = df_sp.loc[df_sp['week_number']>=75]

In [ ]:
dict_metrics = {
    'p':[],
    'd':[],
    'q':[],
    'mape':[],
    'me':[],
    'mae':[],
    'mpe':[],
    'rmse':[],
    'acf1':[],
    'corr':[],
    'minmax':[]
}

i = 0

for p in tqdm(range(10)):
    for d in range(10):
        for q in range(10):
            try:
                modelo_arima = ARIMA(dados_treino['obitosNovosMA'], order=[p,d,q])
                modelo_arima_treinado = modelo_arima.fit()

                # Forecast
                fc = modelo_arima_treinado.forecast(dados_teste.shape[0], alpha=0.05)  # 95% conf

                metrics = forecast_accuracy(fc, dados_teste['obitosNovosMA'])

                dict_metrics['p'].append(p)
                dict_metrics['d'].append(d)
                dict_metrics['q'].append(q)
                dict_metrics['mape'].append(metrics['mape'])
                dict_metrics['me'].append(metrics['me'])
                dict_metrics['mae'].append(metrics['mae'])
                dict_metrics['mpe'].append(metrics['mpe'])
                dict_metrics['rmse'].append(metrics['rmse'])
                dict_metrics['acf1'].append(metrics['acf1'])
                dict_metrics['corr'].append(metrics['corr'])
                dict_metrics['minmax'].append(metrics['minmax'])

            except Exception as e:
                print('Error:', e, 'p:', p, 'd:', d, 'q:', q)

In [ ]:
correls = np.array(dict_metrics['corr'])
mapes = np.array(dict_metrics['mape'])
rmses = np.array(dict_metrics['rmse'])
maes = np.array(dict_metrics['mae'])
list_p = np.array(dict_metrics['p'])
list_d = np.array(dict_metrics['d'])
list_q = np.array(dict_metrics['q'])

nan_indexes = np.argwhere(np.isnan(correls))
correls = np.delete(correls, nan_indexes, axis=0)
mapes = np.delete(mapes, nan_indexes, axis=0)
rmses = np.delete(rmses, nan_indexes, axis=0)
maes = np.delete(maes, nan_indexes, axis=0)
list_p = np.delete(list_p, nan_indexes, axis=0)
list_d = np.delete(list_d, nan_indexes, axis=0)
list_q = np.delete(list_q, nan_indexes, axis=0)

inv_mapes = np.reciprocal(mapes)
inv_rmses = np.reciprocal(rmses)
inv_maes = np.reciprocal(maes)

result = np.multiply(np.multiply(np.multiply(inv_mapes,correls), inv_rmses), inv_maes)

In [ ]:
max_arg = result.argmax(axis=0)
max_arg

In [ ]:
dict_metrics.keys()

In [ ]:
p_max = list_p[max_arg]
d_max = list_d[max_arg]
q_max = list_q[max_arg]

In [ ]:
p_max # 0

In [ ]:
d_max # 0

In [ ]:
q_max # 1

In [ ]:
# modelo_arima = ARIMA(dados_treino['obitosNovos'].values.astype('float32'), order=[p_max,d_max,q_max])
modelo_arima = ARIMA(dados_treino['obitosNovosMA'].values.astype('float32'), order=[8,1,9])
modelo_arima_treinado = modelo_arima.fit()
forecast = modelo_arima_treinado.get_forecast(dados_teste.shape[0])

fc = modelo_arima_treinado.forecast(dados_teste.shape[0], alpha=0.05)
dados_teste['Previsao'] = fc
# dados_teste['Previsao'] = forecast.predicted_mean

# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()

# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_treino.loc[dados_treino['data']>='2021-06-01', 'obitosNovosMA'], label="Train")
plt.plot(dados_teste['obitosNovosMA'], color='orange', label="Test Real")
plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

# plt.figure(figsize=(12,6))
# sns.lineplot(data=dados_treino[250:], x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='Previsao')

In [ ]:
forecast_accuracy(dados_teste['Previsao'], dados_teste['obitosNovosMA'])

### AUTOARIMA

In [ ]:
test

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/wwwusage.csv', names=['value'], header=0)

In [ ]:
previsao_autoarima = modelo_autoarima.predict(n_periods=31)
dados_teste['previsao_autoarima'] = previsao_autoarima

plt.figure(figsize=(12,6))
sns.lineplot(data=dados_treino, x='data', y='R_mean')
sns.lineplot(data=dados_teste, x='data', y='R_mean')
sns.lineplot(data=dados_teste, x='data', y='previsao_autoarima')

O modelo ARIMA se comportou melhor.

In [ ]:
modelo_arima = ARIMA(dados_treino['R_mean'].values.astype('float32'), order=[2,0,0])
modelo_arima_treinado = modelo_arima.fit()
previsao = modelo_arima_treinado.forecast(120)[0]

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(data=previsao)

In [ ]:
previsao_autoarima = modelo_autoarima.predict(n_periods=120)

plt.figure(figsize=(12,6))
sns.lineplot(data=previsao_autoarima)

# SARIMA

## São Paulo

## Predição diária

In [ ]:
dados_treino = df_sp.loc[df_sp['week_number']<75]
dados_teste = df_sp.loc[df_sp['week_number']>=75]

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=df_sp, x='data', y='obitosNovos')
plt.show();

In [ ]:
p = d = q = range(0, 5)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
dict_metrics = {
    'param':[],
    'param_seasonal':[],
    'mape':[],
    'me':[],
    'mae':[],
    'mpe':[],
    'rmse':[],
    'acf1':[],
    'corr':[],
    'minmax':[],
    'aic':[]
}

for param in tqdm(pdq):
    for param_seasonal in seasonal_pdq:
        try:
            model = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'].values,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = model.fit()

            # Forecast
            fc = results.forecast(dados_teste.shape[0], alpha=0.05)  # 95% conf
            fc = pd.Series(fc, index =dados_teste.index)
            metrics = forecast_accuracy(fc, dados_teste['obitosNovos'])

            dict_metrics['param'].append(param)
            dict_metrics['param_seasonal'].append(param_seasonal)
            dict_metrics['mape'].append(metrics['mape'])
            dict_metrics['me'].append(metrics['me'])
            dict_metrics['mae'].append(metrics['mae'])
            dict_metrics['mpe'].append(metrics['mpe'])
            dict_metrics['rmse'].append(metrics['rmse'])
            dict_metrics['acf1'].append(metrics['acf1'])
            dict_metrics['corr'].append(metrics['corr'])
            dict_metrics['minmax'].append(metrics['minmax'])
            dict_metrics['aic'].append(results.aic)

        except Exception as e:
            print('Error:', e, 'param:', param, 'param_seasonal:', param_seasonal)

In [ ]:
correls = np.array(dict_metrics['corr'])
mapes = np.array(dict_metrics['mape'])
rmses = np.array(dict_metrics['rmse'])
maes = np.array(dict_metrics['mae'])
list_param = np.array(dict_metrics['param'])
list_param_seasonal = np.array(dict_metrics['param_seasonal'])
list_aics = np.array(dict_metrics['aic'])

nan_indexes = np.argwhere(np.isnan(correls))
correls = np.delete(correls, nan_indexes, axis=0)
mapes = np.delete(mapes, nan_indexes, axis=0)
rmses = np.delete(rmses, nan_indexes, axis=0)
maes = np.delete(maes, nan_indexes, axis=0)
list_param = np.delete(list_param, nan_indexes, axis=0)
list_param_seasonal = np.delete(list_param_seasonal, nan_indexes, axis=0)
list_aics = np.delete(list_aics, nan_indexes, axis=0)

inv_mapes = np.reciprocal(mapes)
inv_rmses = np.reciprocal(rmses)
inv_maes = np.reciprocal(maes)
inv_aics = np.reciprocal(list_aics)

result = np.multiply(np.multiply(np.multiply(np.multiply(inv_mapes,correls), inv_rmses), inv_maes),inv_aics)

In [ ]:
max_arg = result.argmax(axis=0)
max_arg

In [ ]:
dict_metrics.keys()

In [ ]:
param_max = list_param[max_arg]
param_seasonal_max = list_param_seasonal[max_arg]

In [ ]:
param_max # array([0, 1, 2])

In [ ]:
param_seasonal_max # array([3, 3, 2, 7])

In [ ]:
dados_teste

In [ ]:
# modelo_arima = ARIMA(dados_treino['obitosNovos'].values.astype('float32'), order=[p_max,d_max,q_max])
# model = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'].values,order=param_max,seasonal_order=param_seasonal_max,enforce_stationarity=False,enforce_invertibility=False)
model = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'].values,order=[0,1,2],seasonal_order=[3,3,2,7],enforce_stationarity=False,enforce_invertibility=False)
results = model.fit()
forecast = results.get_forecast(dados_teste.shape[0])

fc = results.forecast(dados_teste.shape[0], alpha=0.05)
dados_teste['Previsao'] = fc
# dados_teste['Previsao'] = forecast.predicted_mean

# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()

# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_treino.loc[dados_treino['data']>='2021-06-01', 'obitosNovos'], label="Train")
plt.plot(dados_teste['obitosNovos'], color='orange', label="Test Real")
plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

# plt.figure(figsize=(12,6))
# sns.lineplot(data=dados_treino[250:], x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='Previsao')

In [ ]:
results.summary()

In [ ]:
# transformação de boxcox
# série transformada, log por exemplo

In [ ]:
forecast_accuracy(dados_teste['Previsao'], dados_teste['obitosNovos'])

### Predicting all dates

In [ ]:
confidence_intervals = np.empty((0,2), int)
predictions = []

for data in df_sp.loc[df_sp['data']>='2020-05-01', 'data'].sort_values():
    print(data)
    dados_treino = df_sp.loc[df_sp['data']<data]
    dados_teste = df_sp.loc[df_sp['data']==data]
    
#     modelo_arima = ARIMA(dados_treino['obitosNovos'].values, order=[p_max,d_max,q_max])
    model = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'].values,order=[0,1,2],seasonal_order=[3,3,2,7],enforce_stationarity=False,enforce_invertibility=False)
    results = model.fit()
    forecast = results.get_forecast(dados_teste.shape[0])

    fc = results.forecast(dados_teste.shape[0], alpha=0.05)
    predictions.append(fc[0])
    # dados_teste['Previsao'] = forecast.predicted_mean

    # Get confidence intervals of forecasts
    confidence_intervals = np.concatenate((confidence_intervals,[forecast.conf_int()[0]]))

# # make series for plotting purpose
# lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
# upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# # Plot
# plt.figure(figsize=(15,6))
# plt.plot(dados_treino['obitosNovos'], label="Train")
# plt.plot(dados_teste['obitosNovos'], color='orange', label="Test Real")
# plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
# plt.fill_between(lower_series.index, 
#                  lower_series, 
#                  upper_series, 
#                  color='k', alpha=.15)

# plt.legend()
# plt.title("Covid Forecasting - Deaths")
# plt.show()

In [ ]:
dados_teste = df_sp.loc[df_sp['data']>='2020-05-01']

In [ ]:
dados_teste.shape

In [ ]:
confidence_intervals.shape

In [ ]:
# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)
df_predictions = pd.Series(predictions, index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_teste[300:400]['obitosNovos'], color='orange', label="Test Real")
plt.plot(df_predictions[300:400], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series[300:400].index, 
                 lower_series[300:400], 
                 upper_series[300:400], 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

## Predição semanal

In [ ]:
df_sp_wk = df_sp.groupby(['week_number'])['obitosNovos'].sum().reset_index()

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=df_sp_wk, x='week_number', y='obitosNovos')
plt.show();

In [ ]:
plot_acf(df_sp_wk['obitosNovos']); #entendendo autocorrelação

In [ ]:
plot_pacf(df_sp_wk['obitosNovos']); #entendendo autocorrelação

In [ ]:
dados_treino = df_sp_wk.loc[df_sp_wk['week_number']<75]
dados_teste = df_sp_wk.loc[df_sp_wk['week_number']>=75]

In [ ]:
p = d = q = range(0, 5)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
dict_metrics = {
    'param':[],
    'param_seasonal':[],
    'mape':[],
    'me':[],
    'mae':[],
    'mpe':[],
    'rmse':[],
    'acf1':[],
    'corr':[],
    'minmax':[],
    'aic':[]
}

for param in tqdm(pdq):
    for param_seasonal in seasonal_pdq:
        try:
            model = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'].values,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = model.fit()

            # Forecast
            fc = results.forecast(dados_teste.shape[0], alpha=0.05)  # 95% conf
            fc = pd.Series(fc, index =dados_teste.index)
            metrics = forecast_accuracy(fc, dados_teste['obitosNovos'])

            dict_metrics['param'].append(param)
            dict_metrics['param_seasonal'].append(param_seasonal)
            dict_metrics['mape'].append(metrics['mape'])
            dict_metrics['me'].append(metrics['me'])
            dict_metrics['mae'].append(metrics['mae'])
            dict_metrics['mpe'].append(metrics['mpe'])
            dict_metrics['rmse'].append(metrics['rmse'])
            dict_metrics['acf1'].append(metrics['acf1'])
            dict_metrics['corr'].append(metrics['corr'])
            dict_metrics['minmax'].append(metrics['minmax'])
            dict_metrics['aic'].append(results.aic)

        except Exception as e:
            print('Error:', e, 'param:', param, 'param_seasonal:', param_seasonal)

In [ ]:
correls = np.array(dict_metrics['corr'])
mapes = np.array(dict_metrics['mape'])
rmses = np.array(dict_metrics['rmse'])
maes = np.array(dict_metrics['mae'])
list_param = np.array(dict_metrics['param'])
list_param_seasonal = np.array(dict_metrics['param_seasonal'])
list_aics = np.array(dict_metrics['aic'])

nan_indexes = np.argwhere(np.isnan(correls))
correls = np.delete(correls, nan_indexes, axis=0)
mapes = np.delete(mapes, nan_indexes, axis=0)
rmses = np.delete(rmses, nan_indexes, axis=0)
maes = np.delete(maes, nan_indexes, axis=0)
list_param = np.delete(list_param, nan_indexes, axis=0)
list_param_seasonal = np.delete(list_param_seasonal, nan_indexes, axis=0)
list_aics = np.delete(list_aics, nan_indexes, axis=0)

inv_mapes = np.reciprocal(mapes)
inv_rmses = np.reciprocal(rmses)
inv_maes = np.reciprocal(maes)
inv_aics = np.reciprocal(list_aics)

result = np.multiply(np.multiply(np.multiply(np.multiply(inv_mapes,correls), inv_rmses), inv_maes),inv_aics)

In [ ]:
max_arg = result.argmax(axis=0)
max_arg

In [ ]:
dict_metrics.keys()

In [ ]:
param_max = list_param[max_arg]
param_seasonal_max = list_param_seasonal[max_arg]

In [ ]:
param_max

In [ ]:
param_seasonal_max

In [ ]:
# modelo_arima = ARIMA(dados_treino['obitosNovos'].values.astype('float32'), order=[p_max,d_max,q_max])
model = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'].values,order=param_max,seasonal_order=param_seasonal_max,enforce_stationarity=False,enforce_invertibility=False)
results = model.fit()
forecast = results.get_forecast(dados_teste.shape[0])

fc = results.forecast(dados_teste.shape[0], alpha=0.05)
dados_teste['Previsao'] = fc
# dados_teste['Previsao'] = forecast.predicted_mean

# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()

# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_treino.loc[dados_treino['week_number']>=50, 'obitosNovos'], label="Train")
plt.plot(dados_teste['obitosNovos'], color='orange', label="Test Real")
plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

# plt.figure(figsize=(12,6))
# sns.lineplot(data=dados_treino[250:], x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='obitosNovos')
# sns.lineplot(data=dados_teste, x='data', y='Previsao')

In [ ]:
forecast_accuracy(dados_teste['Previsao'], dados_teste['obitosNovos'])

## Média móvel - diário

In [ ]:
dados_treino = df_sp.loc[df_sp['week_number']<75]
dados_teste = df_sp.loc[df_sp['week_number']>=75]

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=df_sp, x='data', y='obitosNovosMA')
plt.show();

In [ ]:
p = d = q = range(0, 5)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
dict_metrics = {
    'param':[],
    'param_seasonal':[],
    'mape':[],
    'me':[],
    'mae':[],
    'mpe':[],
    'rmse':[],
    'acf1':[],
    'corr':[],
    'minmax':[],
    'aic':[]
}

i = 0

for param in tqdm(pdq):
    for param_seasonal in seasonal_pdq:
        try:
            model = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovosMA'].values,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = model.fit()

            # Forecast
            fc = results.forecast(dados_teste.shape[0], alpha=0.05)  # 95% conf
            fc = pd.Series(fc, index =dados_teste.index)
            metrics = forecast_accuracy(fc, dados_teste['obitosNovosMA'])

            dict_metrics['param'].append(param)
            dict_metrics['param_seasonal'].append(param_seasonal)
            dict_metrics['mape'].append(metrics['mape'])
            dict_metrics['me'].append(metrics['me'])
            dict_metrics['mae'].append(metrics['mae'])
            dict_metrics['mpe'].append(metrics['mpe'])
            dict_metrics['rmse'].append(metrics['rmse'])
            dict_metrics['acf1'].append(metrics['acf1'])
            dict_metrics['corr'].append(metrics['corr'])
            dict_metrics['minmax'].append(metrics['minmax'])
            dict_metrics['aic'].append(results.aic)

        except Exception as e:
            print('Error:', e, 'param:', param, 'param_seasonal:', param_seasonal)

In [ ]:
correls = np.array(dict_metrics['corr'])
mapes = np.array(dict_metrics['mape'])
rmses = np.array(dict_metrics['rmse'])
maes = np.array(dict_metrics['mae'])
list_param = np.array(dict_metrics['param'])
list_param_seasonal = np.array(dict_metrics['param_seasonal'])
list_aics = np.array(dict_metrics['aic'])

nan_indexes = np.argwhere(np.isnan(correls))
correls = np.delete(correls, nan_indexes, axis=0)
mapes = np.delete(mapes, nan_indexes, axis=0)
rmses = np.delete(rmses, nan_indexes, axis=0)
maes = np.delete(maes, nan_indexes, axis=0)
list_param = np.delete(list_param, nan_indexes, axis=0)
list_param_seasonal = np.delete(list_param_seasonal, nan_indexes, axis=0)
list_aics = np.delete(list_aics, nan_indexes, axis=0)

inv_mapes = np.reciprocal(mapes)
inv_rmses = np.reciprocal(rmses)
inv_maes = np.reciprocal(maes)
inv_aics = np.reciprocal(list_aics)

result = np.multiply(np.multiply(np.multiply(np.multiply(inv_mapes,correls), inv_rmses), inv_maes),inv_aics)

In [ ]:
max_arg = result.argmax(axis=0)
max_arg

In [ ]:
dict_metrics.keys()

In [ ]:
param_max = list_param[max_arg]
param_seasonal_max = list_param_seasonal[max_arg]

In [ ]:
param_max

In [ ]:
param_seasonal_max

In [ ]:
# modelo_arima = ARIMA(dados_treino['obitosNovosMA'].values.astype('float32'), order=[p_max,d_max,q_max])
model = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovosMA'].values,order=param_max,seasonal_order=param_seasonal_max,enforce_stationarity=False,enforce_invertibility=False)
results = model.fit()
forecast = results.get_forecast(dados_teste.shape[0])

fc = results.forecast(dados_teste.shape[0], alpha=0.05)
dados_teste['Previsao'] = fc
# dados_teste['Previsao'] = forecast.predicted_mean

# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()

# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_treino.loc[dados_treino['data']>='2021-06-01', 'obitosNovosMA'], label="Train")
plt.plot(dados_teste['obitosNovosMA'], color='orange', label="Test Real")
plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

# plt.figure(figsize=(12,6))
# sns.lineplot(data=dados_treino[250:], x='data', y='obitosNovosMA')
# sns.lineplot(data=dados_teste, x='data', y='obitosNovosMA')
# sns.lineplot(data=dados_teste, x='data', y='Previsao')

In [ ]:
forecast_accuracy(dados_teste['Previsao'], dados_teste['obitosNovosMA'])

# Arimax

## Predição diária

In [ ]:
df_vacinacao_first.rename(columns={'qtde_vacinas':'nb_vac_first', 'vacina_dataaplicacao':'data'}, inplace=True)
df_vacinacao_second.rename(columns={'qtde_vacinas':'nb_vac_second', 'vacina_dataaplicacao':'data'}, inplace=True)

In [ ]:
df_vacinacao_first['data'] = pd.to_datetime(df_vacinacao_first['data'])
df_vacinacao_second['data'] = pd.to_datetime(df_vacinacao_second['data'])

In [ ]:
df_sp = df_sp.merge(df_vacinacao_first[['data', 'nb_vac_first']], how='left', on='data')
df_sp = df_sp.merge(df_vacinacao_second[['data', 'nb_vac_second']], how='left', on='data')

In [ ]:
df_sp[['nb_vac_first', 'nb_vac_second']] = df_sp[['nb_vac_first', 'nb_vac_second']].fillna(0)

In [ ]:
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))
    
d1 = df_sp['nb_vac_first_MA']
d2 = df_sp['obitosNovosMA']
seconds = 5
fps = 30
rs = [crosscorr(d1,d2, lag) for lag in range(-int(seconds*fps),int(seconds*fps+1))]
offset = np.floor(len(rs)/2)-np.argmax(rs)
f,ax=plt.subplots(figsize=(14,3))
ax.plot(rs)
ax.axvline(np.ceil(len(rs)/2),color='k',linestyle='--',label='Center')
ax.axvline(np.argmax(rs),color='r',linestyle='--',label='Peak synchrony')
ax.set(title=f'Offset = {offset} frames\nS1 leads <> S2 leads',ylim=[-.61,.61],xlim=[0,301], xlabel='Offset',ylabel='Pearson r')
ax.set_xticks([0, 50, 100, 151, 201, 251, 301])
ax.set_xticklabels([-150, -100, -50, 0, 50, 100, 150]);
plt.legend()

In [ ]:
plt.figure(figsize=(15,6))
sns.lineplot(data=df_sp, x='data', y='nb_vac_first_MA_shift')
sns.lineplot(data=df_sp, x='data', y='nb_vac_second_MA')
df_sp['obitosNovosMAx'] = df_sp['obitosNovosMA']*800
sns.lineplot(data=df_sp, x='data', y='obitosNovosMAx')
plt.show();

In [ ]:
dias_media_movel = 7

df_sp['obitosNovosMA'] = df_sp['obitosNovos'].transform(lambda x: x.rolling(dias_media_movel, 1).mean())
df_sp['nb_vac_first_MA'] = df_sp['nb_vac_first'].transform(lambda x: x.rolling(dias_media_movel, 1).mean())
df_sp['nb_vac_second_MA'] = df_sp['nb_vac_second'].transform(lambda x: x.rolling(dias_media_movel, 1).mean())

In [ ]:
corr_list = []

for i in range(1,91):
    df_sp['nb_vac_first_MA_shift'] = df_sp['nb_vac_first_MA'].shift(i)
    corr_list.append(df_sp['nb_vac_first_MA_shift'].corr(df_sp['obitosNovosMA']))

In [ ]:
df_sp['nb_vac_first_shift'] = df_sp['nb_vac_first'].shift(30)
df_sp['nb_vac_second_shift'] = df_sp['nb_vac_second'].shift(30)

In [ ]:
df_sp[['nb_vac_first_shift', 'nb_vac_second_shift']] = df_sp[['nb_vac_first_shift', 'nb_vac_second_shift']].fillna(0)

In [ ]:
dados_treino = df_sp.loc[df_sp['week_number']<75]
dados_teste = df_sp.loc[df_sp['week_number']>=75]

In [ ]:
df_sp.columns

In [ ]:
df_sp.to_csv('../data/external/sao_paulo_obitos_vacina.csv', index=False)

In [ ]:
dict_metrics = {
    'p':[],
    'd':[],
    'q':[],
    'mape':[],
    'me':[],
    'mae':[],
    'mpe':[],
    'rmse':[],
    'acf1':[],
    'corr':[],
    'minmax':[]
}

i = 0

for p in tqdm(range(10)):
    for d in range(10):
        for q in range(10):
            try:
                results = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'],order=(p,d,q),seasonal_order=(0,0,0,0),exog=dados_treino[['nb_vac_first_shift', 'nb_vac_second_shift']],
                                   enforce_stationarity=False, enforce_invertibility=False,).fit()

                # Forecast
                fc = results.forecast(dados_teste.shape[0], alpha=0.05, exog=dados_teste[['nb_vac_first_shift', 'nb_vac_second_shift']])  # 95% conf

                metrics = forecast_accuracy(fc, dados_teste['obitosNovos'])

                dict_metrics['p'].append(p)
                dict_metrics['d'].append(d)
                dict_metrics['q'].append(q)
                dict_metrics['mape'].append(metrics['mape'])
                dict_metrics['me'].append(metrics['me'])
                dict_metrics['mae'].append(metrics['mae'])
                dict_metrics['mpe'].append(metrics['mpe'])
                dict_metrics['rmse'].append(metrics['rmse'])
                dict_metrics['acf1'].append(metrics['acf1'])
                dict_metrics['corr'].append(metrics['corr'])
                dict_metrics['minmax'].append(metrics['minmax'])

            except Exception as e:
                print('Error:', e, 'p:', p, 'd:', d, 'q:', q)

In [ ]:
correls = np.array(dict_metrics['corr'])
mapes = np.array(dict_metrics['mape'])
rmses = np.array(dict_metrics['rmse'])
maes = np.array(dict_metrics['mae'])
list_p = np.array(dict_metrics['p'])
list_d = np.array(dict_metrics['d'])
list_q = np.array(dict_metrics['q'])

nan_indexes = np.argwhere(np.isnan(correls))
correls = np.delete(correls, nan_indexes, axis=0)
mapes = np.delete(mapes, nan_indexes, axis=0)
rmses = np.delete(rmses, nan_indexes, axis=0)
maes = np.delete(maes, nan_indexes, axis=0)
list_p = np.delete(list_p, nan_indexes, axis=0)
list_d = np.delete(list_d, nan_indexes, axis=0)
list_q = np.delete(list_q, nan_indexes, axis=0)

inv_mapes = np.reciprocal(mapes)
inv_rmses = np.reciprocal(rmses)
inv_maes = np.reciprocal(maes)

result = np.multiply(np.multiply(np.multiply(inv_mapes,correls), inv_rmses), inv_maes)

In [ ]:
max_arg = result.argmax(axis=0)
max_arg

In [ ]:
dict_metrics.keys()

In [ ]:
p_max = list_p[max_arg]
d_max = list_d[max_arg]
q_max = list_q[max_arg]

In [ ]:
p_max # 5

In [ ]:
d_max # 1

In [ ]:
q_max # 4

In [ ]:
# modelo_arima = ARIMA(dados_treino['obitosNovosMA'].values.astype('float32'), order=[p_max,d_max,q_max])
# results = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'],order=(p_max,d_max,q_max),seasonal_order=(0,0,0,0),exog=dados_treino[['nb_vac_first_shift', 'nb_vac_second_shift']],
#                                    enforce_stationarity=False, enforce_invertibility=False,).fit()
results = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'],order=(5,1,4),seasonal_order=(0,0,0,0),exog=dados_treino[['nb_vac_first_shift', 'nb_vac_second_shift']],
                                   enforce_stationarity=False, enforce_invertibility=False,).fit()
forecast = results.get_forecast(dados_teste.shape[0], exog = dados_teste[['nb_vac_first_shift', 'nb_vac_second_shift']])

fc = results.forecast(dados_teste.shape[0], alpha=0.05, exog = dados_teste[['nb_vac_first_shift', 'nb_vac_second_shift']])
dados_teste['Previsao'] = fc
# dados_teste['Previsao'] = forecast.predicted_mean

# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()

# make series for plotting purpose
lower_series = pd.Series(confidence_intervals['lower obitosNovos'], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals['upper obitosNovos'], index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_treino.loc[dados_treino['data']>='2021-06-01', 'obitosNovos'], label="Train")
plt.plot(dados_teste['obitosNovos'], color='orange', label="Test Real")
plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()

In [ ]:
forecast_accuracy(dados_teste['Previsao'], dados_teste['obitosNovos'])

In [ ]:
confidence_intervals = np.empty((0,2), int)
predictions = []

for data in df_sp.loc[df_sp['data']>='2020-05-01', 'data'].sort_values():
    print(data)
    dados_treino = df_sp.loc[df_sp['data']<data]
    dados_teste = df_sp.loc[df_sp['data']==data]
    
#     modelo_arima = ARIMA(dados_treino['obitosNovos'].values, order=[p_max,d_max,q_max])
    results = sm.tsa.statespace.SARIMAX(dados_treino['obitosNovos'],order=(5,1,4),seasonal_order=(0,0,0,0),exog=dados_treino[['nb_vac_first_shift', 'nb_vac_second_shift']],
                                   enforce_stationarity=False, enforce_invertibility=False,).fit()
    forecast = results.get_forecast(dados_teste.shape[0], exog = dados_teste[['nb_vac_first_shift', 'nb_vac_second_shift']])

    fc = results.forecast(dados_teste.shape[0], alpha=0.05, exog = dados_teste[['nb_vac_first_shift', 'nb_vac_second_shift']])
    predictions.append(fc.iloc[0])
    # dados_teste['Previsao'] = forecast.predicted_mean

    conf_int = forecast.conf_int()
    conf_int = conf_int.to_numpy()[0]
    
    # Get confidence intervals of forecasts
    confidence_intervals = np.concatenate((confidence_intervals,[conf_int]))

# # make series for plotting purpose
# lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
# upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)

# # Plot
# plt.figure(figsize=(15,6))
# plt.plot(dados_treino['obitosNovos'], label="Train")
# plt.plot(dados_teste['obitosNovos'], color='orange', label="Test Real")
# plt.plot(dados_teste['Previsao'], color='darkgreen', label="Test Forecast")
# plt.fill_between(lower_series.index, 
#                  lower_series, 
#                  upper_series, 
#                  color='k', alpha=.15)

# plt.legend()
# plt.title("Covid Forecasting - Deaths")
# plt.show()

In [ ]:
dados_teste = df_sp.loc[df_sp['data']>='2020-05-01']

In [ ]:
dados_teste.shape

In [ ]:
confidence_intervals.shape

In [ ]:
# make series for plotting purpose
lower_series = pd.Series(confidence_intervals[:, 0], index=dados_teste.index)
upper_series = pd.Series(confidence_intervals[:, 1], index=dados_teste.index)
df_predictions = pd.Series(predictions, index=dados_teste.index)

# Plot
plt.figure(figsize=(15,6))
plt.plot(dados_teste[300:400]['obitosNovos'], color='orange', label="Test Real")
plt.plot(df_predictions[300:400], color='darkgreen', label="Test Forecast")
plt.fill_between(lower_series[300:400].index, 
                 lower_series[300:400], 
                 upper_series[300:400], 
                 color='k', alpha=.15)

plt.legend()
plt.title("Covid Forecasting - Deaths")
plt.show()